In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Customer Churn Prediction using Neural Networks

## Overview
This notebook demonstrates the process of building, training, and evaluating a neural network model for predicting customer churn. We use an augmented dataset for training and validation, and a separate test set for final evaluation.

## Dataset
- Training data: "../data/augmented-training data.csv"
- Test data: "../customer_churn_dataset-testing-master.csv"

## Key Steps
1. Data Loading and Preprocessing
   - Load training and test datasets
   - Split training data into train and validation sets
   - Scale features using StandardScaler

2. Model Architecture
   - Sequential model with Dense layers
   - Batch Normalization and Dropout for regularization
   - Binary classification with sigmoid activation

3. Model Training
   - Use Adam optimizer and binary crossentropy loss
   - Train for 50 epochs with early stopping

4. Model Evaluation
   - Evaluate on separate test set
   - Generate classification report and confusion matrix

5. Visualizations
   - Plot training and validation accuracy/loss curves

## Requirements
- Python 3.x
- Libraries: pandas, numpy, sklearn, tensorflow, matplotlib, seaborn

## Usage
Run the cells in order to load data, build the model, train, and evaluate results. Adjust hyperparameters as needed for optimization.

## Results
The notebook will output:
- Model accuracy and loss on test set
- Classification report with precision, recall, and F1-score
- Confusion matrix visualization
- Training history plots

In [ ]:
# Step 1: Load and preprocess the data
def load_and_preprocess_data(train_file, test_file):
    """
    Load the CSV files and preprocess the data.
    """
    # Load the CSV files
    train_df = pd.read_csv(train_file)
    test_df = pd.read_csv(test_file)
    
    # Separate features and target for training data
    X_train = train_df.drop('Churn', axis=1)
    y_train = train_df['Churn']
    
    # Separate features and target for test data
    X_test = test_df.drop('Churn', axis=1)
    y_test = test_df['Churn']
    
    # Split the training data into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
    
    # Scale the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    X_test_scaled = scaler.transform(X_test)
    
    return X_train_scaled, X_val_scaled, X_test_scaled, y_train, y_val, y_test


In [ ]:
# Step 2: Build the neural network model
def build_model(input_shape):
    """
    Build and compile the neural network model.
    """
    model = keras.Sequential([
        keras.layers.Dense(64, activation='relu', input_shape=(input_shape,)),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(32, activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(16, activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [ ]:
# Step 3: Train the model
def train_model(model, X_train, y_train, X_test, y_test, epochs=50, batch_size=32):
    """
    Train the model and return the training history.
    """
    history = model.fit(X_train, y_train,
                        epochs=epochs,
                        batch_size=batch_size,
                        validation_data=(X_test, y_test),
                        verbose=1)
    return history


In [ ]:
# Step 4: Evaluate the model
def evaluate_model(model, X_test, y_test):
    """
    Evaluate the model and print the results.
    """
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test accuracy: {accuracy:.4f}")
    print(f"Test loss: {loss:.4f}")
    
    # Make predictions
    y_pred = model.predict(X_test)
    y_pred_classes = (y_pred > 0.5).astype(int).reshape(-1)
    
    # Print classification report
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred_classes))
    
    # Create confusion matrix
    cm = confusion_matrix(y_test, y_pred_classes)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title('Confusion Matrix')
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.show()

In [ ]:
# Step 5: Plot training history
def plot_training_history(history):
    """
    Plot the training and validation accuracy and loss.
    """
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.tight_layout()
    plt.show()


In [ ]:
# Load and preprocess the data
X_train, X_val, X_test, y_train, y_val, y_test = load_and_preprocess_data(
        "../data/augmented-training data.csv",
        "../customer_churn_dataset-testing-master.csv"
    )

# Build the model
model = build_model(X_train.shape[1])

# Train the model
history = train_model(model, X_train, y_train, X_test, y_test)

# Evaluate the model
evaluate_model(model, X_test, y_test)

# Plot training history
plot_training_history(history)